In [1]:
!nvidia-smi 

Fri Mar 29 14:43:57 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   38C    P0    36W / 250W |     35MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  Off  | 00000000:5E:00.0 Off |                    0 |
| N/A   

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/app


In [3]:
!pip install ultralytics==8.0.196

In [4]:
from IPython import display
display.clear_output()


In [5]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.8.10 torch-2.2.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40354MiB)
Setup complete ✅ (72 CPUs, 250.5 GB RAM, 2708.1/3266.8 GB disk)


In [6]:
from ultralytics import YOLO
from IPython.display import display, Image

In [7]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="CtzMCbHKOEOCfDHjdqVX")
# project = rf.workspace("franz-bpzvh").project("license-ocr-qqq6v")
# dataset = project.version(2).download("yolov8")

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="CtzMCbHKOEOCfDHjdqVX")
project = rf.workspace("franz-bpzvh").project("license-ocr-qqq6v")
dataset = project.version(3).download("yolov8")

# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="2GV100bGHy4A1eirlRCf")
# project = rf.workspace("sarsoteam").project("charenyeni")
# version = project.version(8)
# dataset = version.download("yolov8")


/app/datasets
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to license-ocr-3 in yolov8:: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 39666/39666 [00:04<00:00, 9626.13it/s]


In [9]:
!yolo mode=train model=yolov8x.pt data=/app/datasets/license-ocr-3/data.yaml epochs=100 imgsz=640 plots=True save_period=-1 batch=64 device=0,1,2,3

New https://pypi.org/project/ultralytics/8.1.37 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.8.10 torch-2.2.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40354MiB)
                                                            CUDA:1 (NVIDIA A100-PCIE-40GB, 40354MiB)
                                                            CUDA:2 (NVIDIA A100-PCIE-40GB, 40354MiB)
                                                            CUDA:3 (NVIDIA A100-PCIE-40GB, 40354MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/app/datasets/license-ocr-3/data.yaml, epochs=100, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=(0, 1, 2, 3), workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4,

In [10]:
import zipfile
import os

def zip_folder(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))

folder_to_zip = '/app/datasets/runs'
zip_file_name = 'yolov8x_stats.zip'
zip_file_path = os.path.join(os.getcwd(), zip_file_name)

zip_folder(folder_to_zip, zip_file_path)

print(f"Folder '{folder_to_zip}' zipped successfully to '{zip_file_path}'")

Folder '/app/datasets/runs' zipped successfully to '/app/datasets/yolov8x_stats.zip'


In [ ]:
!yolo task=detect mode=predict model=/app/datasets/runs/detect/train/weights/best.pt source={dataset.location}/test/images save=True